# 카카오맵 별점 가져오기

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 

import tqdm


In [2]:
res_df = pd.read_excel('combined_file1_22.xlsx')
res_df = res_df.loc[res_df['검색여부']=='가능',:]
res_df.reset_index(inplace=True)
res_df.drop(['index','Unnamed: 0'],axis=1,inplace=True )

In [3]:
res_df.head()

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,사업장명_원본,검색여부,index_original
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길 18,",정식당,가능,0
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로 378,",채선당 샤브보트 강남역점,가능,1
2,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로 343,",수아당,가능,3
3,영업/정상,서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",알찬한끼,분식,203453.074369,443736.685792,서울특별시,강남구,"역삼로 204,",알찬한끼,가능,5
4,영업/정상,서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",세계관,한식,204952.006872,445195.026116,서울특별시,강남구,"삼성로96길 11,",세계관,가능,7


In [4]:
kakao_star_df = pd.DataFrame
kakao_star_list = []

In [5]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
# driver = webdriver.Chrome(ChromeDriverManager().install())

# 주소이름 
url = 'https://map.kakao.com/'

driver.get(url)

In [6]:
# 카카오 별점 가져오는 함수 

for i in range(0,len(res_df)) :
    # 검색어 = 지역명 + 가게명 ex) 강남구 떡도리탕
    name = res_df['상세주소'][i] + ' ' + res_df['사업장명'][i]
    
    # 검색 결과가 있을 때
    try :               
        time.sleep(3)
        WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.ID,'search.keyword.query')))
        # 검색 박스 비우고 검색어 입력하고 enter
        driver.find_element(By.ID,'search.keyword.query').clear()
        driver.find_element(By.ID,'search.keyword.query').send_keys(name)
        driver.find_element(By.ID,'search.keyword.query').send_keys(Keys.RETURN)

        # 2초 대기
        time.sleep(2)

        # 별점 저장
        kakao_star = driver.find_element(By.XPATH,'//*[@id="info.search.place.list"]/li[1]/div[4]/span[1]/em').text
        kakao_star = float(kakao_star)
        kakao_star_list.append(kakao_star)
    # 검색 결과가 없을 때
    except :
        kakao_star = '없음' 

    print('-----------------------------')
    print('식당 이름:',name)
    print('별점:',kakao_star)

-----------------------------
식당 이름: 강남대로114길 18, 정식당
별점: 없음
-----------------------------
식당 이름: 강남대로 378, 채선당 샤브보트
별점: 4.0
-----------------------------
식당 이름: 압구정로 343, 수아당
별점: 0.0
-----------------------------
식당 이름: 역삼로 204, 알찬한끼
별점: 0.0
-----------------------------
식당 이름: 삼성로96길 11, 세계관
별점: 0.0
-----------------------------
식당 이름: 역삼로 409, 분식을 품다
별점: 5.0
-----------------------------
식당 이름: 논현로146길 11, 식당 민홍
별점: 5.0
-----------------------------
식당 이름: 논현로157길 55, 디어
별점: 5.0
-----------------------------
식당 이름: 개포로 506, 버거리
별점: 5.0
-----------------------------
식당 이름: 논현로8길 11, 제주그집
별점: 5.0
-----------------------------
식당 이름: 강남대로 628, 만수정
별점: 5.0
-----------------------------
식당 이름: 언주로134길 11, 은서네 국내산 대패삼겹살
별점: 0.0
-----------------------------
식당 이름: 삼성로 517, 냉동고집
별점: 없음
-----------------------------
식당 이름: 역삼로 220, 리하베스트
별점: 0.0
-----------------------------
식당 이름: 논현로76길 12, 팡뮤제
별점: 5.0
-----------------------------
식당 이름: 논현로159길 10, 만소당
별점: 4.6
--------------------------